# Loading Weather Data
### Loading Forecasts data from Microsoft Planetary Computer Catalogue
 
ECMWF Open Data (real-time) From Microsoft Planetary Computer Dataset.

https://planetarycomputer.microsoft.com/dataset/ecmwf-forecast#overview




### Install the eecodes C library and cartopy native (C/C++) dependencies

In [ ]:
%conda install -c conda-forge eccodes cartopy

### Install other dependencies

In [ ]:
%pip install cfgrib eccodes rioxarray pystac_client planetary-computer

In [ ]:
import numpy as np
import urllib.request
import eccodes
import cfgrib
import rioxarray
import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import pyspark.sql.functions as f


---

### Query the STAC Planetary Computer data catalogue for ECMWF Forecast data.

https://github.com/stac-api-extensions/query

https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/

Find ``enfo`` and ``waef`` products. Search by ``forecast_datetime`` for tomorrow. 

In [ ]:
import pystac_client
import planetary_computer
from datetime import datetime, timedelta

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)


forecastdt = (datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%SZ')

search = catalog.search(
    collections=["ecmwf-forecast"],
    query={
        "ecmwf:stream": {"in": ["oper", "wave"]},
        "ecmwf:type": {"eq": "fc"},
        "ecmwf:forecast_datetime": {"eq": forecastdt}
    },
)
items = search.item_collection()
items


### Find the latest grib files for wave and atmospheric forecasts

In [ ]:
def findLatestForecast(items, stream):
    stream_items = list(filter(lambda x: x.properties["ecmwf:stream"] == stream, items))
    #Latest forecast date
    item_latest_date = max(stream_items, key=lambda item: item.properties["ecmwf:forecast_datetime"]).properties["ecmwf:forecast_datetime"]
    #Smallest step (most recent model run)
    item = min(filter(lambda x: x.properties["ecmwf:forecast_datetime"] == item_latest_date, stream_items), key=lambda item: item.properties["ecmwf:step"] )
    return item

oper = findLatestForecast(items, "oper")
wave = findLatestForecast(items, "wave")


### Download gribs from Planetary Computer

Saving in OneLake

In [ ]:
def downloadForecast(item):
    url = item.assets["data"].href
    id = item.id
    filename = f"/lakehouse/default/Files/{id}.grib"
    print(f"Downloading {filename}")
    urllib.request.urlretrieve(url,filename)
    urllib.request.urlcleanup()
    return(filename)

operFile = downloadForecast(oper)
waveFile = downloadForecast(wave)


### Load and combine using xarray

In [ ]:
#filter to t2m (temperature - 2 metres)
ds_temps = xr.open_dataset(operFile, engine="cfgrib", filter_by_keys={'typeOfLevel': 'heightAboveGround','level':2.0})
ds_waves = xr.open_dataset(waveFile, engine="cfgrib")

ds = xr.merge( [ds_temps, ds_waves])
ds


## Clip to approx Western Australia coast

In [ ]:
#very approx bounding box for Western Australia
min_lat = -20
min_lon = 110
max_lat = -36
max_lon = 120


ds = ds.sel(latitude=slice(min_lat, max_lat), longitude=slice(min_lon,max_lon))
ds["t2m"] = ds.t2m - 273
ds


## Let's plot the t2m data

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.Mercator())
ax.coastlines(resolution="10m")
plot = ds.t2m.plot(
    cmap=plt.cm.coolwarm, transform=ccrs.PlateCarree()
)

#### We can interpolate like this

https://docs.xarray.dev/en/stable/user-guide/interpolation.html

In [ ]:
new_lon = np.linspace(ds.longitude[0], ds.longitude[-1], ds.dims["longitude"] * 25)
new_lat = np.linspace(ds.latitude[0], ds.latitude[-1], ds.dims["latitude"] * 25)
ds_i = ds.interp(latitude=new_lat, longitude=new_lon)

#### Plot both OG and lerp'd data

In [ ]:
projection = ccrs.Mercator()
fig, axes = plt.subplots(ncols=2,figsize=(15,10), subplot_kw=dict(projection=projection))
axes[0].coastlines(resolution='10m')
axes[1].coastlines(resolution='10m')

axes[0].set_title("Raw data")
axes[1].set_title("Interpolated data")

ds.t2m.plot(ax=axes[0], transform=ccrs.PlateCarree());
ds_i.t2m.plot(ax=axes[1], transform=ccrs.PlateCarree());


### Extract to dataframe

In [ ]:
weather_df = ds["t2m"].to_dataframe().reset_index()
weather_df

### Clean

In [ ]:
weather_df.drop(['step', 'heightAboveGround','meanSea'], axis=1, inplace=True)
weather_df.rename(columns={'time': 'forecastMadeDate', 'valid_time': 'forecastForDate', 't2m': 'temperature'},  inplace=True)
weather_df.drop_duplicates()

### Save to Lakehouse

In [ ]:
table_name = "weatherModel"

df = spark.createDataFrame(weather_df)
df.write.mode("overwrite").format("delta").option("overwriteSchema", "True").save(f"Tables/{table_name}")


### Next...
From here, we could load the ``shipwrecks`` and spatial join to the nearest forecast point, or we could map our new model forecasts into BOM maritime regions and use the existing ``shipwrecks`` to ``forecasts`` relationship, or... or...

https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.sjoin_nearest.html


In [ ]:
%%sql

SELECT * FROM weatherModel LIMIT 100